In [31]:
import csv
import re
import os
from collections import defaultdict

# 디렉토리 및 출력 파일 경로 설정
log_dir = "./log_data"
csv_output_path = "./h5_summary.csv"

# 변경된 로그 파일명 패턴 (trial 추가)
filename_pattern = r"run(?P<trial>\d+)_alpha(?P<alpha>[0-9.]+)_preclude(?P<preclude>\d+)_preserve(?P<preserve>\d+)_temp(?P<temp>\w+)\.log"

# 수집할 RocksDB 통계 항목
target_keys = [
    'rocksdb.flush.write.bytes',
    'rocksdb.compact.write.bytes',
    'rocksdb.bytes.written',
    'rocksdb.compaction.total.time.cpu_micros',
    'rocksdb.bytes.compressed.from',
    'rocksdb.bytes.compressed.to',
    'rocksdb.wal.bytes',
    'rocksdb.stall.micros',
    'rocksdb.memtable.payload.bytes.at.flush',
    'rocksdb.memtable.garbage.bytes.at.flush',
    'rocksdb.last.level.read.bytes',
    'rocksdb.last.level.read.count',
    'rocksdb.non.last.level.read.bytes',
    'rocksdb.non.last.level.read.count',
]

# 지연 시간 필드
latency_fields = [
    'write.P50', 'write.P95', 'write.P99', 'write.P100', 'write.COUNT', 'write.SUM', 'write.AVG',
]

# SST 레벨 필드: 총 파일 수와 각 온도별 개수 포함
sst_level_fields = []
for i in range(7):
    sst_level_fields.append(f"L{i}_total")
    for temp_type in ["Hot", "Warm", "Cold", "Unknown"]:
        sst_level_fields.append(f"L{i}_{temp_type}")

# 최종 CSV 헤더 (trial 포함)
header = ['trial', 'alpha', 'preclude', 'preserve', 'temp', 'time(s)', 'keys', 'db_size_bytes'] + \
         sst_level_fields + target_keys + latency_fields

rows = []

# 로그 파일 순회
for log_file_name in os.listdir(log_dir):
    if not log_file_name.endswith('.log'):
        continue

    match = re.match(filename_pattern, log_file_name)
    if not match:
        print(f"⚠️ Filename {log_file_name} does not match the expected pattern.")
        continue

    trial = int(match.group("trial"))
    alpha = float(match.group("alpha"))
    preclude = int(match.group("preclude"))
    preserve = int(match.group("preserve"))
    temp = match.group("temp")

    log_path = os.path.join(log_dir, log_file_name)

    # 초기값 설정
    stats = {key: 0 for key in target_keys}
    latency = {key: 0 for key in latency_fields}
    sst_files = defaultdict(lambda: {"total": 0, "Hot": 0, "Warm": 0, "Cold": 0, "Unknown": 0})
    time_sec = 0
    keys = 0
    db_size_bytes = 0
    current_level = None
    in_sst_block = False

    with open(log_path, "r") as f:
        for line in f:
            line = line.strip()

            # 시간, 키 수, DB 크기 추출
            if (m := re.match(r"⏱️ Total write time: ([\d\.]+) seconds", line)):
                time_sec = float(m.group(1))
            elif (m := re.match(r"\[💾 Writing (\d+) keys\.\.\.\]", line)):
                keys = int(m.group(1))
            elif (m := re.match(r"DB 디렉토리 사용량: (\d+) bytes", line)):
                db_size_bytes = int(m.group(1))

            # SST 블록 시작 및 종료
            elif line.startswith("===== Level별 SST 파일 개수 및 온도별 분포 ====="):
                in_sst_block = True
                continue
            elif line.startswith("==============================================="):
                in_sst_block = False
                current_level = None
                continue

            elif in_sst_block:
                # Level 추출
                if (m := re.match(r"Level (\d+) ?: ?총 (\d+)개 파일", line)):
                    current_level = int(m.group(1))
                    total_files = int(m.group(2))
                    sst_files[current_level]['total'] = total_files
                # 온도별 개수 추출
                elif (m := re.match(r"(Hot|Warm|Cold|Unknown) ?: ?(\d+)개", line)):
                    temp_type, count = m.group(1), int(m.group(2))
                    if current_level is not None:
                        sst_files[current_level][temp_type] = count

            # RocksDB 통계
            elif (m := re.match(r"(rocksdb\.[\w\.]+)\s+COUNT\s*:\s*(\d+)", line)):
                key, value = m.group(1), int(m.group(2))
                if key in stats:
                    stats[key] = value

            # write latency
            elif (m := re.match(
                r"rocksdb\.db\.write\.micros\s+P50\s*:\s*([\d\.]+)\s+P95\s*:\s*([\d\.]+)\s+P99\s*:\s*([\d\.]+)\s+P100\s*:\s*([\d\.]+)\s+COUNT\s*:\s*(\d+)\s+SUM\s*:\s*(\d+)", line)):
                p50, p95, p99, p100, count, total = map(float, m.groups())
                latency.update({
                    'write.P50': p50, 'write.P95': p95, 'write.P99': p99, 'write.P100': p100,
                    'write.COUNT': int(count), 'write.SUM': int(total),
                    'write.AVG': total / count if count > 0 else 0
                })

    # SST 데이터 행 만들기
    sst_row = []
    for i in range(7):
        level_info = sst_files[i]
        sst_row.append(level_info['total'])
        for temp_type in ['Hot', 'Warm', 'Cold', 'Unknown']:
            sst_row.append(level_info[temp_type])

    # 한 줄 완성
    row = [trial, alpha, preclude, preserve, temp, time_sec, keys, db_size_bytes] + \
          sst_row + \
          [stats[key] for key in target_keys] + \
          [latency[key] for key in latency_fields]

    rows.append(row)

# CSV 저장
with open(csv_output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(rows)

print(f"✅ CSV summary saved to {csv_output_path}")

✅ CSV summary saved to ./h5_summary.csv


In [32]:
import pandas as pd
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.max_rows', None)  # 모든 행 표시

df = pd.read_csv("./h5_summary.csv")
df

,trial,alpha,preclude,preserve,temp,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,rocksdb.flush.write.bytes,rocksdb.compact.write.bytes,rocksdb.bytes.written,rocksdb.compaction.total.time.cpu_micros,rocksdb.bytes.compressed.from,rocksdb.bytes.compressed.to,rocksdb.wal.bytes,rocksdb.stall.micros,rocksdb.memtable.payload.bytes.at.flush,rocksdb.memtable.garbage.bytes.at.flush,rocksdb.last.level.read.bytes,rocksdb.last.level.read.count,rocksdb.non.last.level.read.bytes,rocksdb.non.last.level.read.count,write.P50,write.P95,write.P99,write.P100,write.COUNT,write.SUM,write.AVG
0,3,0.9,120,120,cold,80.1453,1000000,490190093,3,0,0,0,3,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,619767064,1703799646,16409090136,26965667,46540665295,2311989748,16409090136,65395351,16272153227,3897962679,178011283,213761,1798659055,2187474,9.981431,19.140115,30.754212,1166823.0,1000000,78007249,78.007249
1,3,0.9,15,15,cold,81.2269,1000000,445166284,4,0,0,0,4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,3,0,620305599,1948591586,16409090136,32185839,51489950820,2556404263,16409090136,66026921,16272153227,3887204988,656939965,797769,1626986966,1979925,9.825643,20.024498,53.498608,1240045.0,1000000,78985278,78.985278
2,2,0.9,120,120,cold,74.9174,1000000,529706241,5,0,0,0,5,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,620601117,1671862993,16409090136,26485410,45942118884,2281735990,16409090136,61477760,16272153227,3881301358,190868108,230022,1769127584,2151760,8.920343,14.893800,28.061768,877218.0,1000000,72874334,72.874334
3,1,0.5,0,0,cold,89.6692,1000000,655261253,15,0,0,0,15,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,3,6,0,0,6,0,813780014,2386743086,16410542657,36191711,64058397638,3181678696,16410542657,73683571,16339479239,106416754,336932638,405172,2282380671,2773244,9.752324,20.047564,64.238527,1000505.0,1000000,87247336,87.247336
4,1,0.9,15,15,cold,69.0993,1000000,485861920,4,0,0,0,4,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,2,0,619719243,1855441977,16409090136,29588557,49607078393,2463144339,16409090136,54779063,16272153227,3898913852,544013787,660570,1607179019,1954316,9.443313,19.105810,39.457963,1545619.0,1000000,66941579,66.941579
5,2,1.2,120,120,cold,72.1720,1000000,226581221,2,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,235451945,746445468,16407152794,12772972,19698573251,977583325,16407152794,59000817,16270230905,11566890722,63135066,76106,833049959,1015725,9.007107,17.973036,30.336723,934197.0,1000000,70328549,70.328549
6,1,0.5,20,20,cold,100.7980,1000000,775259920,7,0,0,0,7,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,0,2,2,0,0,0,2,4,0,0,4,0,810412971,2821357416,16410542657,42698498,72764396696,3613679313,16410542657,84500331,16273594301,107761563,921271217,1115954,2147684411,2611275,10.308270,20.543086,1259.136646,1237233.0,1000000,98286288,98.286288
7,3,0.9,40,40,cold,86.3098,1000000,425254101,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,2,0,0,2,0,620601240,1965036073,16409090136,31601729,51810091141,2573190544,16409090136,71792052,16272153227,3881301358,517266995,626047,1784426591,2172161,9.633992,18.767742,40.772085,1538023.0,1000000,84075140,84.075140
8,1,1.2,120,120,cold,68.0847,1000000,291311751,11,0,0,0,11,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,235456821,762647827,16407152794,13360403,19995682630,992478553,16407152794,54855779,16270230905,11566792323,106324034,128375,819666642,999808,9.141868,18.177456,31.613937,759635.0,1000000,66221754,66.221754
9,2,1.2,25,25,cold,64.2916,1000000,217187600,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0,234949426,954852127,16407152794,15537420,23862270737,1184071451,16407152794,50893545,1627

In [38]:
import pandas as pd

# 데이터 복사
df = pd.read_csv("./h5_summary.csv")

# === 1. 주요 지표 계산 === #

# WAF (Write Amplification Factor)
df['WAF'] = df.apply(
    lambda row: (
        (row.get('rocksdb.wal.bytes', 0) +
         row.get('rocksdb.flush.write.bytes', 0) +
         row.get('rocksdb.compact.write.bytes', 0))
        / row['rocksdb.bytes.written']
        if row['rocksdb.bytes.written'] != 0 else 0
    ),
    axis=1
)

# Throughput (Bytes Written / Time)
df['throughput'] = df.apply(
    lambda row: (
        row['rocksdb.bytes.written'] / row['time(s)']
        if (row.get('rocksdb.bytes.written', 0) > 0 and row.get('time(s)', 0) > 0)
        else 0
    ),
    axis=1
)

# Stall Time 계산 수정: row.get() 사용
df['stall_flush_total(us)'] = df.apply(
    lambda row: (
        row.get('rocksdb.stall.micros', 0)
    ),
    axis=1
)

# last level read ratio
df['last_level_byte_ratio'] = df.apply(
    lambda row: (
        row.get('rocksdb.last.level.read.bytes', 0) /
        (row.get('rocksdb.last.level.read.bytes', 0) + row.get('rocksdb.non.last.level.read.bytes', 0))
        if (row.get('rocksdb.last.level.read.bytes', 0) + row.get('rocksdb.non.last.level.read.bytes', 0)) > 0
        else 0
    ),
    axis=1
)

df['last_level_count_ratio'] = df.apply(
    lambda row: (
        row.get('rocksdb.last.level.read.count', 0) /
        (row.get('rocksdb.last.level.read.count', 0) + row.get('rocksdb.non.last.level.read.count', 0))
        if (row.get('rocksdb.last.level.read.count', 0) + row.get('rocksdb.non.last.level.read.count', 0)) > 0
        else 0
    ),
    axis=1
)

# alpha, preserve 조합에 따른 case 매핑 함수
def map_case(alpha, preserve):
    if alpha == 0.5:
        mapping = {0: 0, 20: 20, 50: 50, 80: 80, 120: 100}
    elif alpha == 0.9:
        mapping = {0: 0, 15: 20, 40: 50, 65: 80, 120: 100}
    elif alpha == 1.2:
        mapping = {0: 0, 10: 20, 25: 50, 40: 80, 120: 100}
    else:
        return None
    return mapping.get(preserve, None)

# === 2. Compaction / Flush / Stall 관련 분석 === #

analysis_rows = []

for idx, row in df.iterrows():
    total_time = row['time(s)']
    compaction_time = row.get('rocksdb.compaction.total.time.cpu_micros', 0) / 1e6
    compaction_ratio = compaction_time / total_time if total_time > 0 else 0

    stall_micros = row.get('rocksdb.stall.micros', 0)
    stall_flush_total = stall_micros / 1e6

    compressed_from = row.get('rocksdb.bytes.compressed.from', 0)
    compressed_to = row.get('rocksdb.bytes.compressed.to', 0)
    flush_bytes = row.get('rocksdb.flush.write.bytes', 0)
    compact_bytes = row.get('rocksdb.compact.write.bytes', 0)
    latency = row.get('write.AVG', 0)

    # case 계산
    case = map_case(row['alpha'], row['preserve'])

    analysis_rows.append({
        "trial": row["trial"],
        "alpha": row["alpha"],
        "preclude": row["preclude"],
        "preserve": row["preserve"],
        "case": case,
        "time(s)": total_time,
        "keys": row["keys"],
        "db_size_bytes": row["db_size_bytes"],
        
        "L0_total": row["L0_total"],
        "L0_Hot": row["L0_Hot"],
        "L0_Warm": row["L0_Warm"],
        "L0_Cold": row["L0_Cold"],
        "L0_Unknown": row["L0_Unknown"],

        "L1_total": row["L1_total"],
        "L1_Hot": row["L1_Hot"],
        "L1_Warm": row["L1_Warm"],
        "L1_Cold": row["L1_Cold"],
        "L1_Unknown": row["L1_Unknown"],

        "L2_total": row["L2_total"],
        "L2_Hot": row["L2_Hot"],
        "L2_Warm": row["L2_Warm"],
        "L2_Cold": row["L2_Cold"],
        "L2_Unknown": row["L2_Unknown"],

        "L3_total": row["L3_total"],
        "L3_Hot": row["L3_Hot"],
        "L3_Warm": row["L3_Warm"],
        "L3_Cold": row["L3_Cold"],
        "L3_Unknown": row["L3_Unknown"],

        "L4_total": row["L4_total"],
        "L4_Hot": row["L4_Hot"],
        "L4_Warm": row["L4_Warm"],
        "L4_Cold": row["L4_Cold"],
        "L4_Unknown": row["L4_Unknown"],

        "L5_total": row["L5_total"],
        "L5_Hot": row["L5_Hot"],
        "L5_Warm": row["L5_Warm"],
        "L5_Cold": row["L5_Cold"],
        "L5_Unknown": row["L5_Unknown"],

        "L6_total": row["L6_total"],
        "L6_Hot": row["L6_Hot"],
        "L6_Warm": row["L6_Warm"],
        "L6_Cold": row["L6_Cold"],
        "L6_Unknown": row["L6_Unknown"],

        "WAF": row["WAF"],
        "throughput": row["throughput"],
        "last_level_byte_ratio": row["last_level_byte_ratio"],
        "last_level_count_ratio": row["last_level_count_ratio"],
        "compaction_time(s)": compaction_time,
        "compaction_ratio": round(compaction_ratio, 4),
        "stall_flush_total(s)": stall_flush_total,
        "Latency": latency,
        "compressed_from": compressed_from,
        "compressed_to": compressed_to,
        "flush_bytes": flush_bytes,
        "compact_bytes": compact_bytes,
    })

# NaN 값 제거
df.fillna(0, inplace=True)

# 분석 결과 DataFrame 생성
df_analysis = pd.DataFrame(analysis_rows)

# 결과 저장
df_analysis.to_csv('modified_h5_summary.csv', index=False)


In [39]:
df_analysis

,trial,alpha,preclude,preserve,case,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,WAF,throughput,last_level_byte_ratio,last_level_count_ratio,compaction_time(s),compaction_ratio,stall_flush_total(s),Latency,compressed_from,compressed_to,flush_bytes,compact_bytes
0,3,0.9,120,120,100,80.1453,1000000,490190093,3,0,0,0,3,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.141602,2.047418e+08,0.090056,0.089021,26.965667,0.3365,65.395351,78.007249,46540665295,2311989748,619767064,1703799646
1,3,0.9,15,15,20,81.2269,1000000,445166284,4,0,0,0,4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,3,0,1.156553,2.020155e+08,0.287636,0.287206,32.185839,0.3962,66.026921,78.985278,51489950820,2556404263,620305599,1948591586
2,2,0.9,120,120,100,74.9174,1000000,529706241,5,0,0,0,5,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.139707,2.190291e+08,0.097382,0.096576,26.485410,0.3535,61.477760,72.874334,45942118884,2281735990,620601117,1671862993
3,1,0.5,0,0,0,89.6692,1000000,655261253,15,0,0,0,15,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,3,6,0,0,6,0,1.195028,1.830120e+08,0.128634,0.127476,36.191711,0.4036,73.683571,87.247336,64058397638,3181678696,813780014,2386743086
4,1,0.9,15,15,20,69.0993,1000000,485861920,4,0,0,0,4,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,2,0,1.150841,2.374711e+08,0.252889,0.252619,29.588557,0.4282,54.779063,66.941579,49607078393,2463144339,619719243,1855441977
5,2,1.2,120,120,100,72.1720,1000000,226581221,2,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1.059846,2.273340e+08,0.070449,0.069705,12.772972,0.1770,59.000817,70.328549,19698573251,977583325,235451945,746445468
6,1,0.5,20,20,20,100.7980,1000000,775259920,7,0,0,0,7,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,0,2,2,0,0,0,2,4,0,0,4,0,1.221307,1.628062e+08,0.300190,0.299406,42.698498,0.4236,84.500331,98.286288,72764396696,3613679313,810412971,2821357416
7,3,0.9,40,40,50,86.3098,1000000,425254101,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,2,0,0,2,0,1.157573,1.901185e+08,0.224733,0.223731,31.601729,0.3661,71.792052,84.075140,51810091141,2573190544,620601240,1965036073
8,1,1.2,120,120,100,68.0847,1000000,291311751,11,0,0,0,11,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1.060834,2.409815e+08,0.114822,0.113789,13.360403,0.1962,54.855779,66.221754,19995682630,992478553,235456821,762647827
9,2,1.2,25,25,50,64.2916,1000000,217187600,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0,1.072517,2.551990e+08,0.373829,0.373246,15.537420,0.2417,50.893545,62.490198,23862270737,1184071451,234949426,954852127


In [40]:
df_analysis.groupby(['alpha', 'preclude', 'preserve']).max().reset_index()

,alpha,preclude,preserve,trial,case,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,WAF,throughput,last_level_byte_ratio,last_level_count_ratio,compaction_time(s),compaction_ratio,stall_flush_total(s),Latency,compressed_from,compressed_to,flush_bytes,compact_bytes
0,0.5,0,0,3,0,94.5538,1000000,771675114,15,0,0,0,15,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,6,0,0,6,0,1.198810,1.830120e+08,0.128634,0.127476,37.554767,0.4045,78.489013,92.122487,65343754345,3245575368,813780014,2452121820
1,0.5,20,20,3,20,107.0370,1000000,818805517,7,0,0,0,7,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,3,0,0,0,3,4,0,0,4,0,1.234920,1.754605e+08,0.347710,0.347174,46.375950,0.4339,90.747457,104.464685,77221097245,3835225666,810508380,3044784893
2,0.5,50,50,3,50,96.2382,1000000,981680731,16,0,0,0,16,1,0,0,0,1,2,0,0,0,2,2,0,0,0,2,3,0,0,0,3,4,0,0,0,4,3,0,0,3,0,1.218218,2.404662e+08,0.282798,0.282038,42.075325,0.4594,81.718556,93.902719,71742264022,3563701606,813743995,2770654615
3,0.5,80,80,3,80,92.9991,1000000,863567522,11,0,0,0,11,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,6,0,0,0,6,0,0,0,0,0,1.197548,2.469173e+08,0.143668,0.142052,37.286730,0.4807,77.619021,90.737078,64917116588,3226282314,810508356,2431498444
4,0.5,120,120,3,100,91.1497,1000000,838716997,13,0,0,0,13,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,6,0,0,0,6,0,0,0,0,0,1.193996,2.466260e+08,0.150303,0.148693,36.592611,0.4967,75.673877,88.675808,63763219099,3168525436,810508267,2373072481
5,0.9,0,0,3,0,75.5422,1000000,547019144,7,0,0,0,7,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,4,0,0,4,0,1.139101,2.927304e+08,0.103687,0.102581,27.301129,0.4616,60.871668,73.321471,45736191687,2270695114,620594438,1661932503
6,0.9,15,15,3,20,81.2269,1000000,485861920,4,0,0,0,4,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,3,0,1.167073,2.374711e+08,0.387034,0.386504,33.310000,0.4322,66.026921,78.985278,54932808813,2726878227,620601213,2120914407
7,0.9,40,40,3,50,93.6986,1000000,479577568,3,0,0,0,3,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,2,0,0,2,0,1.182150,2.504088e+08,0.406325,0.405734,37.093508,0.4013,78.932042,91.444470,59867913937,2972771135,620601240,2368889984
8,0.9,65,65,3,80,81.0613,1000000,496911833,7,0,0,0,7,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.144088,2.472749e+08,0.096282,0.094986,27.555144,0.3971,67.010713,78.977339,47380447085,2353440737,620304729,1744582453
9,0.9,120,120,3,100,80.1453,1000000,529706241,5,0,0,0,5,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.141602,2.190291e+08,0.097382,0.096576,27.127565,0.3565,65.395351,78.007249,46540665295,2311989748,620601117,1703799646


In [41]:
df_analysis.groupby(['alpha', 'preclude', 'preserve']).min().reset_index()

,alpha,preclude,preserve,trial,case,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,WAF,throughput,last_level_byte_ratio,last_level_count_ratio,compaction_time(s),compaction_ratio,stall_flush_total(s),Latency,compressed_from,compressed_to,flush_bytes,compact_bytes
0,0.5,0,0,1,0,89.6692,1000000,655261253,4,0,0,0,4,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,6,0,0,6,0,1.195028,1.735577e+08,0.118704,0.117441,36.191711,0.3972,73.683571,87.247336,64058397638,3181678696,810458613,2386743086
1,0.5,20,20,1,20,93.5284,1000000,714172861,4,0,0,0,4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,0,2,4,0,0,4,0,1.213589,1.533165e+08,0.215606,0.215258,40.579383,0.4236,78.678092,91.071974,70209996041,3487543958,810387378,2694607846
2,0.5,50,50,1,50,68.2447,1000000,767770112,5,0,0,0,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,4,0,0,0,4,0,0,0,0,0,1.175604,1.705200e+08,0.063655,0.062445,31.349235,0.4069,53.878359,65.851186,57684532897,2867293594,810418356,2071279101
3,0.5,80,80,1,80,66.4617,1000000,735577171,4,0,0,0,4,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,4,0,0,0,4,0,0,0,0,0,1.179023,1.764592e+08,0.063810,0.062556,31.946739,0.4009,52.209387,64.093152,58828158546,2924083673,810373861,2127363341
4,0.5,120,120,1,100,66.5402,1000000,718693060,5,0,0,0,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,4,0,0,0,4,0,0,0,0,0,1.181956,1.800395e+08,0.114570,0.113433,33.051157,0.4015,51.802798,64.241660,59799960273,2972051525,810482490,2175513432
5,0.9,0,0,1,0,56.0553,1000000,442607665,4,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,3,0,0,3,0,1.133066,2.172175e+08,0.097492,0.096745,25.112105,0.3614,41.942264,53.856583,43773355015,2172681355,620309435,1562899626
6,0.9,15,15,1,20,69.0993,1000000,429154040,4,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,2,0,1.150841,2.020155e+08,0.252889,0.252619,29.588557,0.3962,54.779063,66.941579,49607078393,2463144339,619719243,1855441977
7,0.9,40,40,1,50,65.5292,1000000,423155022,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,0,0,0,0,0,1.136483,1.751263e+08,0.085368,0.084202,26.299266,0.3661,51.226521,63.447288,44861108380,2228346506,620011515,1619547182
8,0.9,65,65,1,80,66.3597,1000000,459920528,2,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,3,0,0,0,3,0,0,0,0,0,1.136134,2.024282e+08,0.083507,0.082530,26.353643,0.3399,51.757488,64.192492,44740719370,2222502232,619774953,1613842371
9,0.9,120,120,1,100,74.9174,1000000,468635690,3,0,0,0,3,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.138720,2.047418e+08,0.090056,0.089021,26.485410,0.3365,61.477760,72.874334,45617118764,2266037673,619767064,1655961468


In [42]:
df_analysis.groupby(['alpha', 'preclude', 'preserve']).mean().reset_index()

,alpha,preclude,preserve,trial,case,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,WAF,throughput,last_level_byte_ratio,last_level_count_ratio,compaction_time(s),compaction_ratio,stall_flush_total(s),Latency,compressed_from,compressed_to,flush_bytes,compact_bytes
0,0.5,0,0,2.0,0.0,91.982967,1000000.0,7.205374e+08,8.333333,0.0,0.0,0.0,8.333333,1.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,1.000000,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,2.333333,0.0,0.0,0.0,2.333333,6.000000,0.0,0.0,6.000000,0.0,1.197309,1.784928e+08,0.123084,0.121868,36.950575,0.401767,75.931667,89.584361,6.483609e+10,3.220349e+09,8.115800e+08,2.426365e+09
1,0.5,20,20,2.0,20.0,100.454467,1000000.0,7.694128e+08,5.666667,0.0,0.0,0.0,5.666667,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,1.666667,0.0,0.0,0.0,1.666667,2.333333,0.0,0.0,0.0,2.333333,2.333333,0.0,0.0,0.0,2.333333,4.000000,0.0,0.0,4.000000,0.0,1.223272,1.638611e+08,0.287835,0.287279,43.217944,0.430267,84.641960,97.940982,7.339850e+10,3.645483e+09,8.104362e+08,2.853583e+09
2,0.5,50,50,2.0,50.0,84.441200,1000000.0,8.553033e+08,9.000000,0.0,0.0,0.0,9.000000,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,1.666667,0.0,0.0,0.0,1.666667,2.666667,0.0,0.0,0.0,2.666667,4.000000,0.0,0.0,0.0,4.000000,1.000000,0.0,0.0,1.000000,0.0,1.196149,1.985683e+08,0.149824,0.149069,36.526046,0.434500,69.985011,82.082825,6.444537e+10,3.202325e+09,8.115453e+08,2.407374e+09
3,0.5,80,80,2.0,80.0,82.082800,1000000.0,7.812257e+08,8.000000,0.0,0.0,0.0,8.000000,1.000000,0.0,0.0,0.0,1.000000,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,2.333333,0.0,0.0,0.0,2.333333,5.333333,0.0,0.0,0.0,5.333333,0.000000,0.0,0.0,0.000000,0.0,1.188898,2.041550e+08,0.107778,0.106272,35.008781,0.431333,67.367461,79.808189,6.207402e+10,3.085172e+09,8.104635e+08,2.289451e+09
4,0.5,120,120,2.0,100.0,77.477200,1000000.0,7.664324e+08,8.333333,0.0,0.0,0.0,8.333333,0.666667,0.0,0.0,0.0,0.666667,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,2.333333,0.0,0.0,0.0,2.333333,5.333333,0.0,0.0,0.0,5.333333,0.000000,0.0,0.0,0.000000,0.0,1.188898,2.154096e+08,0.137984,0.136510,34.978467,0.456800,62.566196,75.119659,6.207732e+10,3.085133e+09,8.104997e+08,2.289426e+09
5,0.9,0,0,2.0,0.0,63.717367,1000000.0,4.871440e+08,5.000000,0.0,0.0,0.0,5.000000,0.000000,0.0,0.0,0.0,0.000000,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.666667,0.0,0.0,0.0,1.666667,3.333333,0.0,0.0,3.333333,0.0,1.135268,2.618260e+08,0.101574,0.100597,26.096518,0.414900,49.384187,61.553092,4.448801e+10,2.208498e+09,6.204994e+08,1.599117e+09
6,0.9,15,15,2.0,20.0,75.797800,1000000.0,4.533941e+08,4.000000,0.0,0.0,0.0,4.000000,0.333333,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.666667,0.0,0.0,0.0,1.666667,2.666667,0.0,0.0,2.666667,0.0,1.158156,2.174686e+08,0.309186,0.308776,31.694799,0.418867,60.946513,73.598330,5.200995e+10,2.582142e+09,6.202087e+08,1.974983e+09
7,0.9,40,40,2.0,50.0,81.845867,1000000.0,4.426622e+08,2.333333,0.0,0.0,0.0,2.333333,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,2.333333,0.0,0.0,0.0,2.333333,1.333333,0.0,0.0,1.333333,0.0,1.158735,2.052179e+08,0.238809,0.237889,31.664834,0.387767,67.316872,79.655633,5.217970e+10,2.591436e+09,6.202119e+08,1.984491e+09
8,0.9,65,65,2.0,80.0,74.839367,1000000.0,4.819227e+08,4.333333,0.0,0.0,0.0,4.333333,0.333333,0.0,0.0,0.0,0.333333,1.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.666667,0.0,0.0,0.0,1.666667,3.000000,0.0,0.0,0.0,3.000000,0.000000,0.0,0.0,0.000000,0.0,1.140110,2.208466e+08,0.089667,0.